Importing the dependencies

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Device Configuration

In [ ]:
# Make device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

Data Collection and Preprocessing

In [ ]:
data=load_breast_cancer()
X=data.data
Y=data.target

In [ ]:
X

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [ ]:
Y[:5]

array([0, 0, 0, 0, 0])


Splitting the data into Training and Test datasets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(X.shape)
print (X_train.shape)
print (X_test.shape)

(569, 30)
(455, 30)
(114, 30)


In [ ]:
# standardize the data using standardscaler
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [ ]:
type(X_train)

numpy.ndarray

In [ ]:
# convert data to pytorch tensors and move into gpu
X_train=torch.tensor(X_train,dtype=torch.float32).to(device)
X_test=torch.tensor(X_test,dtype=torch.float32).to(device)
y_train=torch.tensor(y_train,dtype=torch.float32).to(device)
y_test=torch.tensor(y_test,dtype=torch.float32).to(device)

Neural Network Architecture

In [ ]:
# 1. Construct a model class that subclasses nn.Module
from torch import nn

# Build model
class NeuralNet(nn.Module):
    def __init__(self, input_features, output_features, hidden_units):
        super().__init__()
        self.linear_layer_stack = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=output_features), # how many classes are there?
        )

    def forward(self, x):
        return self.linear_layer_stack(x)



In [ ]:

model = NeuralNet(input_features=X_train.shape[1],
                    output_features=1,
                    hidden_units=64).to(device)
model

NeuralNet(
  (linear_layer_stack): Sequential(
    (0): Linear(in_features=30, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [ ]:
X_train.shape[1]

30

In [ ]:
# Make predictions with the model
untrained_preds = model(X_test.to(device))
print(f"Length of predictions: {len(untrained_preds)}, Shape: {untrained_preds.shape}")
print(f"Length of test samples: {len(y_test)}, Shape: {y_test.shape}")
print(f"\nFirst 10 predictions:\n{untrained_preds[:10]}")
print(f"\nFirst 10 test labels:\n{y_test[:10]}")

Length of predictions: 114, Shape: torch.Size([114, 1])
Length of test samples: 114, Shape: torch.Size([114])

First 10 predictions:
tensor([[-0.0703],
        [ 0.0441],
        [-0.0034],
        [-0.0384],
        [-0.0043],
        [ 0.0754],
        [ 0.0044],
        [-0.0212],
        [-0.0342],
        [-0.1310]], device='cuda:0', grad_fn=<SliceBackward0>)

First 10 test labels:
tensor([1., 0., 0., 1., 1., 0., 0., 0., 1., 1.], device='cuda:0')


The number of predictions matches the number of test labels, but their shapes or formats are different. We need to change the predictions to match the test labels.

** Setup loss function and optimizer**

In [ ]:
# Create a loss function
# loss_fn = nn.BCELoss() # BCELoss = no sigmoid built-in
loss_fn = nn.BCEWithLogitsLoss() # BCEWithLogitsLoss = sigmoid built-in

# Create an optimizer
optimizer = torch.optim.SGD(params=model.parameters(),
                            lr=0.1)

In [ ]:
# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

Building a training and testing loop

In [ ]:
torch.manual_seed(42)
epochs = 100

# Put data to target device
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

# Build training and evaluation loop
for epoch in range(epochs):
    ### Training
    model.train()

    # 1. Forward pass (model outputs raw logits)same device
    y_logits = model(X_train).squeeze() # squeeze to remove extra `1` dimensions, this won't work unless model and data are on same device
    y_pred = torch.round(torch.sigmoid(y_logits))

    # 2. Calculate loss/accuracy
    # loss = loss_fn(torch.sigmoid(y_logits), # Using nn.BCELoss you need torch.sigmoid()
    #                y_train)
    loss = loss_fn(y_logits, # Using nn.BCEWithLogitsLoss works with raw logits
                   y_train)
    acc = accuracy_fn(y_true=y_train,
                      y_pred=y_pred)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()   #update to the model's parameters

    ### Testing
    model.eval()
    with torch.inference_mode():
        # 1. Forward pass
        test_logits = model(X_test).squeeze()
        test_pred = torch.round(torch.sigmoid(test_logits))
        # 2. Caculate loss/accuracy
        test_loss = loss_fn(test_logits,
                            y_test)
        test_acc = accuracy_fn(y_true=y_test,
                               y_pred=test_pred)

    # Print out what's happening every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} |Train Loss: {loss:.5f},Train Accuracy: {acc:.2f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")

Epoch: 0 |Train Loss: 0.08443,Train Accuracy: 98.02% | Test loss: 0.07708, Test acc: 98.25%
Epoch: 10 |Train Loss: 0.07880,Train Accuracy: 98.02% | Test loss: 0.07376, Test acc: 98.25%
Epoch: 20 |Train Loss: 0.07410,Train Accuracy: 98.02% | Test loss: 0.07128, Test acc: 98.25%
Epoch: 30 |Train Loss: 0.07013,Train Accuracy: 98.24% | Test loss: 0.06930, Test acc: 98.25%
Epoch: 40 |Train Loss: 0.06665,Train Accuracy: 98.24% | Test loss: 0.06766, Test acc: 98.25%
Epoch: 50 |Train Loss: 0.06361,Train Accuracy: 98.68% | Test loss: 0.06634, Test acc: 98.25%
Epoch: 60 |Train Loss: 0.06094,Train Accuracy: 98.68% | Test loss: 0.06527, Test acc: 98.25%
Epoch: 70 |Train Loss: 0.05850,Train Accuracy: 98.68% | Test loss: 0.06439, Test acc: 98.25%
Epoch: 80 |Train Loss: 0.05628,Train Accuracy: 98.68% | Test loss: 0.06369, Test acc: 98.25%
Epoch: 90 |Train Loss: 0.05427,Train Accuracy: 98.68% | Test loss: 0.06312, Test acc: 98.25%


Both The Train accuracy and Test accuracy are good

In [ ]:
model(X_train).squeeze()

tensor([  5.3158, -10.7344,   7.2091,   2.8982,   6.1192,  -5.1027,   4.4613,
          5.4579,   3.9618,  -4.5581,   1.9404,  -6.2140,  -4.1703,   5.1437,
          4.6126,  -7.1758, -10.4688,  -6.7505,   3.2987,   4.4907,   3.4810,
         -4.6427,   1.1140,   4.6092,   0.5025,  -3.2754,   4.3190,  -7.6933,
          7.0305,   7.1292,  -2.8378,   4.0836,  -5.2456,  -6.3849,  -0.6588,
          5.5650,  -1.0916,   6.1837,   8.0237,   2.7645,   4.6398, -14.4808,
         -4.8797,   7.3227,   3.3968,   7.1054,   8.3573,   5.7409,   6.7332,
          4.1903,  -1.0509,   5.0493,   5.1473,   0.3755,   6.6104,  -3.2829,
          3.5319,   4.4305,   4.9575,   0.5100,   4.6615,   2.2167,  -0.9645,
          2.2564,   4.5722,   3.0335,   1.5607,   3.2000,   6.1529,  -2.1672,
         -4.2838,  -2.3645,  -1.2768,   3.0861,  -2.4808,   4.0537,  -6.6943,
          3.7373,   4.1487,   6.8110,   4.5578,  -1.4664,   1.2626,   7.0541,
         -7.5703,   6.5084,   4.0146,   6.6044, -10.3889,   4.17

 Remove Extra Dimension: Sometimes, the model's output has an extra dimension of size 1 that we don't need. .squeeze() removes this unnecessary dimension to match the expected shape of the labels.

Making a Predictive System

In [ ]:

y_pred_labels = torch.round(torch.sigmoid(model(X_test.to(device))[:10]))
y_preds.squeeze()

tensor([1., 0., 0., 1., 1., 0., 0., 0., 1., 1.], device='cuda:0',
       grad_fn=<SqueezeBackward0>)

If y_pred_probs >= 0.5, y=1 (class 1)

If y_pred_probs < 0.5, y=0 (class 0)

In [ ]:
y_test[:10]

tensor([1., 0., 0., 1., 1., 0., 0., 0., 1., 1.], device='cuda:0')

Now it looks like our model's predictions are in the same form as our truth labels (y_test).


Conclusion:

Building a neural network for breast cancer prediction using deep learning involves carefully training and evaluating the model. In our implementation, we achieved a training accuracy of 98.68% and a test accuracy of  98.25%, demonstrating the model's capability to generalize well to unseen data. This indicates that our model can effectively predict breast cancer cases based on the given dataset.